In [1]:
%%CONN /dev/ttyUSB0 115200

Connecting to Serial(/dev/ttyUSB0, 115200)
ready

In [2]:
%%FILE pca9685.py

import ustruct
import time


class PCA9685:
    def __init__(self, i2c, address=0x40):
        self.i2c = i2c
        self.address = address
        self.reset()

    def _write(self, address, value):
        self.i2c.writeto_mem(self.address, address, bytearray([value]))

    def _read(self, address):
        return self.i2c.readfrom_mem(self.address, address, 1)[0]

    def reset(self):
        self._write(0x00, 0x00) # Mode1

    def freq(self, freq=None):
        if freq is None:
            return int(25000000.0 / 4096 / (self._read(0xfe) - 0.5))
        prescale = int(25000000.0 / 4096.0 / freq + 0.5)
        old_mode = self._read(0x00) # Mode 1
        self._write(0x00, (old_mode & 0x7F) | 0x10) # Mode 1, sleep
        self._write(0xfe, prescale) # Prescale
        self._write(0x00, old_mode) # Mode 1
        time.sleep_us(5)
        self._write(0x00, old_mode | 0xa1) # Mode 1, autoincrement on

    def pwm(self, index, on=None, off=None):
        if on is None or off is None:
            data = self.i2c.readfrom_mem(self.address, 0x06 + 4 * index, 4)
            return ustruct.unpack('<HH', data)
        data = ustruct.pack('<HH', on, off)
        self.i2c.writeto_mem(self.address, 0x06 + 4 * index,  data)

    def duty(self, index, value=None, invert=False):
        if value is None:
            pwm = self.pwm(index)
            if pwm == (0, 4096):
                value = 0
            elif pwm == (4096, 0):
                value = 4095
            value = pwm[1]
            if invert:
                value = 4095 - value
            return value
        if not 0 <= value <= 4095:
            raise ValueError("Out of range")
        if invert:
            value = 4095 - value
        if value == 0:
            self.pwm(index, 0, 4096)
        elif value == 4095:
            self.pwm(index, 4096, 0)
        else:
            self.pwm(index, 0, value)



59 lines sent

In [2]:
import os
print(os.listdir())

['boot.py', 'webrepl_cfg.py', 'pca9685.py', 'mcode.py']


In [3]:
import machine, time, math
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
print(i2c.scan())

[64, 112]


In [4]:
import pca9685
c = pca9685.PCA9685(i2c)
c.freq(50)

In [5]:
def ang(g):  # 0.5  mid 1.5  high 2.5
    return((0.5 + g/180*2.0)/(1000/c.freq())*4096)

In [7]:
cpos = []

In [60]:
d = 0.001*2
def m01(v0, v1, v2):
    if not cpos:
        cpos.extend([v0, v1, v2])
        n = 1
    else:
        n = max(abs(v0 - cpos[0]), abs(v1 - cpos[1]), abs(v2 - cpos[2]))
    for i in range(1, n+1):
        c.duty(0, int(cpos[0] + (i/n)*(v0 - cpos[0])))
        c.duty(1, int(cpos[1] + (i/n)*(v1 - cpos[1])))
        c.duty(2, int(cpos[2] + (i/n)*(v2 - cpos[2])))
        time.sleep(d)
    cpos[0], cpos[1], cpos[2] = v0, v1, v2

def m01d(v0, v1, v2):
    c.duty(0, int(v0))
    c.duty(1, int(v1))
    c.duty(2, int(v2))
    cpos[0], cpos[1], cpos[2] = v0, v1, v2


In [9]:

# servo1 has angle perp to arm at 120; subtract to tilt down 
# servo2 has angle upright at 105; subtract to tilt forwards
# servo3 along board at 130; subtract to turn clockwise
# height of servo2 axle is 37mm
# servo2 to servo1 axle is 73mm
# pencil length 150mm

In [10]:
m01(ang(118), ang(138), ang(130))

In [16]:
a0 = 138  # servo2 arm vertical
b0 = 118  # servo1 pencil horizontal

h = 43.8  # height servo2 axle
s = 67.4  # servo1 to 2 axle dist
p = 150   # pencil length
e = 39.7  # servo3 to pencil centre dist

hps1 = -h**2 + (p+s)**2
hps2 = h**2 - (p-s)**2
hps3 = (h-s)**2 - p**2

minx = math.sqrt(p**2 - (h+s)**2)
maxx = math.sqrt(hps1)
print("xrange", minx, maxx)

m01(ang(b0), ang(a0), ang(130))

xrange 100.671 212.942


In [42]:
m01(ang(b0), ang(a0), ang(150))

In [32]:
# a is tilt of s21 from vertical, b is angle of p from horizontal
# h + s*cos(a) = p*sin(b)
# x = s*sin(a) + p*cos(b)

def solveab(x):
    x2 = x**2
    a = 2*math.atan((2*s*x - math.sqrt((hps1 - x2)*(hps2 + x2)))/(hps3 + x2))
    b = math.acos((-s*math.sin(a) + x)/p)
    return math.degrees(a), math.degrees(b)


In [44]:
r0, t0 = 120, 150
a, b = solveab(r0)
m01(ang(b0-(b-a)), ang(a0-a), ang(t0))

In [57]:
def xy(x, y, bplus=0):
    rlsq = (e+x)**2 + (r0+y)**2
    r = math.sqrt(rlsq - e**2)
    t = math.degrees(math.atan(r/e) - math.atan((r0+y)/(e+x)))
    a, b = solveab(r)
    m01d(ang(b0-(b-a)+bplus), ang(a0-a), ang(t0-t))
                     

In [61]:
# steps through at 1mm steps and sleeps between to handle feedrate
feedrate = 100  # mm/sec
xypos = [ 0, 0 ]
def xyf(x, y, bplus=0):
    n = max(abs(x-xypos[0]), abs(y-xypos[1]), 1)
    for i in range(0, n+1):
        l = i/n
        xy(xypos[0]*(1-l) + x*l, xypos[1]*(1-l) + y*l, bplus)
        time.sleep(1/feedrate)
    xypos[0], xypos[1] = x, y

In [66]:
xyf(0,-20)

In [88]:
feedrate = 200
x0 = 0
y0 = -20
for i in range(5):
    xyf(x0+10,y0+10)
    xyf(x0+10,y0-10, 15)
    xyf(x0+-10,y0-10)
    xyf(x0+-10,y0+10)
    xyf(x0+10,y0+10)



In [87]:
feedrate = 100
for i in range(-20, 22, 2):
    xyf(x0-i,y0+i, 15)
    xyf(x0-i+20,y0+i+20, 0)


In [112]:
feedrate = 30
xyf(x0-30,y0+40-30, 0)
xyf(x0,y0+40, 0)

time.sleep(1)
xyf(x0+30,y0+40-30, 0)
xyf(x0,y0+40, 0)


In [113]:
feedrate = 200
for i in range(0, 22, 2):
    xyf(x0+20+i,y0-20+i, 10)
    xyf(x0+i-20,y0+i+20, 0)

    

In [ ]:
for cr in range(5, 35, 5):
    for i in range(260):
        xy(20+math.sin(i*0.2)*cr, math.cos(i*0.2)*cr)
